In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv


In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys

In [3]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [4]:
from transformers import BertTokenizer, TFBertModel
tokenizer =BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [5]:
train2 = train1[1:10000]

In [6]:
sentences = train2.comment_text.values
labels = train2.toxic.values

In [7]:
valid2 = valid[1:1000]

In [8]:
sentences_valid = valid2.comment_text.values
labels_valid = valid2.toxic.values

In [9]:
from tqdm import tqdm

In [10]:
input_ids = []
for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = False) # Construct attn. masks.
                        #return_tensors = 'tf')  
    input_ids.append(encoded_dict['input_ids'])

100%|██████████| 9999/9999 [00:30<00:00, 329.51it/s]


In [11]:
input_ids_valid = []
for sent in tqdm(sentences_valid):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = False) # Construct attn. masks.
                        #return_tensors = 'tf')  
    input_ids_valid.append(encoded_dict['input_ids'])

100%|██████████| 999/999 [00:02<00:00, 352.52it/s]


In [19]:
dataset = tf.data.Dataset.from_tensor_slices((input_ids,labels)).shuffle(10).batch(64).repeat()

In [20]:
dataset_valid = tf.data.Dataset.from_tensor_slices((input_ids_valid,labels_valid)).batch(64)

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [15]:
# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [16]:
with strategy.scope():
    bert_model = TFBertModel.from_pretrained("bert-base-multilingual-cased")
    input_word_ids = Input(shape=(512,), dtype=tf.int32, name="input_word_ids")
    sequence_output = bert_model(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    pre_out=Dense(192,activation = "relu")(cls_token)
    out = Dense(1, activation='sigmoid')(pre_out)
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 512)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 512, 768), (None, 177853440 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 192)               147648    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 193       
Total params: 178,001,281
Trainable params: 178,001,281
Non-trainable params: 0
_________________________________________________________________


In [21]:
train_history = model.fit(dataset,steps_per_epoch=200,epochs=10,validation_data=dataset_valid,verbose=1)

Train for 200 steps, validate for 16 steps
Epoch 1/10
200/200 [==============================] - 185s 924ms/step - loss: 0.2009 - accuracy: 0.9259 - val_loss: 0.3700 - val_accuracy: 0.8338
Epoch 2/10
200/200 [==============================] - 49s 245ms/step - loss: 0.1093 - accuracy: 0.9573 - val_loss: 0.4205 - val_accuracy: 0.8318
Epoch 3/10
200/200 [==============================] - 49s 244ms/step - loss: 0.0658 - accuracy: 0.9754 - val_loss: 0.5288 - val_accuracy: 0.8298
Epoch 4/10
200/200 [==============================] - 49s 243ms/step - loss: 0.0488 - accuracy: 0.9819 - val_loss: 0.4046 - val_accuracy: 0.8238
Epoch 5/10
200/200 [==============================] - 49s 243ms/step - loss: 0.0405 - accuracy: 0.9848 - val_loss: 0.4364 - val_accuracy: 0.8408
Epoch 6/10
200/200 [==============================] - 49s 245ms/step - loss: 0.0300 - accuracy: 0.9879 - val_loss: 0.5143 - val_accuracy: 0.8378
Epoch 7/10
200/200 [==============================] - 49s 243ms/step - loss: 0.0165 - 

In [22]:
train_history.history